# Data Exploration

<b> Data Collection -  </b>
We are trying to fetch toronto places data with the Foursquare places API. Additionaly we are also fetching geocordinates  and toronto postal code data from the below links.

Links
- Geospatial Data - https://cocl.us/Geospatial_data
- Canada postal code data - https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [8]:
import pandas as pd

In [9]:
df2=pd.read_csv("Geospatial_Coordinates.csv")
df2.head(25)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [10]:
import requests
from bs4 import BeautifulSoup
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" #URL
data  = requests.get(url).text 
soup = BeautifulSoup(data,"html5lib") 


contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    field = {}
    if row.span.text=='Not assigned': #Ignore fields with a borough that is Not assigned
        pass
    else:
        field['PostalCode'] = row.p.text[:3]
        field['Borough'] = (row.span.text).split('(')[0]
        field['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        contents.append(field) #combined into one row with the neighborhoods separated with a comma


df=pd.DataFrame(contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [11]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [12]:
newdf = pd.merge(df,df2,left_on='PostalCode',right_on='Postal Code')
newdf = newdf.drop('Postal Code', axis = 1)

Merged Dataframe with postalcode, borough, neighborhood, latitude and longitude

In [6]:
newdf

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Getting FourSquare Data

Plotting neighborhoods of Toronto

In [27]:
!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values


! pip install folium==0.5.0
import folium # plotting library
from folium.plugins import FastMarkerCluster
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.cm as cm

import matplotlib.colors as colors


address = 'Toronto, Ontario, Canada'


geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
tlongitude = longitude

tlatitude = latitude


map_toronto = folium.Map(location=[latitude,longitude],zoom_start=10)


for lat,lng,borough,neighborhood in zip(newdf['Latitude'],newdf['Longitude'],newdf['Borough'],newdf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)


map_toronto

distutils: c:\users\het\appdata\local\programs\python\python39\Include\UNKNOWN
sysconfig: c:\users\het\appdata\local\programs\python\python39\Include
user = False
home = None
root = None
prefix = None
distutils: c:\users\het\appdata\local\programs\python\python39\Include\UNKNOWN
sysconfig: c:\users\het\appdata\local\programs\python\python39\Include
user = False
home = None
root = None
prefix = None
You should consider upgrading via the 'c:\users\het\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


distutils: c:\users\het\appdata\local\programs\python\python39\Include\UNKNOWN
sysconfig: c:\users\het\appdata\local\programs\python\python39\Include
user = False
home = None
root = None
prefix = None
distutils: c:\users\het\appdata\local\programs\python\python39\Include\UNKNOWN
sysconfig: c:\users\het\appdata\local\programs\python\python39\Include
user = False
home = None
root = None
prefix = None
You should consider upgrading via the 'c:\users\het\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [75]:
CLIENT_ID = 'WX5EXQ1VNDEKSRIG14POIVVY3MKTN555UHBIMTD5MYH0ZZQF' # your Foursquare ID
CLIENT_SECRET = 'GY4YKFYU4J31AFSFC3EVGM0RZZZK42C4KILZLSQSYIMXTMFH' # your Foursquare Secret
ACCESS_TOKEN = 'fsq3daN1NA7XlU6QvMrgyqCe4mLWWXHDbhJ5uoO2cEPfepA=' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 50

Making an API Call

In [76]:
# Getting Places data around deifned radius
import requests
radius =3000
headers = {
    "Accept": "application/json",
    "Authorization": "fsq3daN1NA7XlU6QvMrgyqCe4mLWWXHDbhJ5uoO2cEPfepA="
}
url = 'https://api.foursquare.com/v3/places/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    '43.6532', #Downtown Toronto
   '-79.3832', 
    radius, 
   LIMIT)
results = requests.get(url,headers=headers).json()

In [77]:
len(results['results'])

50

Since each call fetches only 50 rows, we are running the loop for 200 times to get more data and keeping the radius as 3km.

In [78]:
import random
for i in range(200):
    Latitude = round(random.uniform(43.6024137,43.8361247),5)
    Longitude = round(random.uniform(-79.615819,-79.1604971),5)
    print(i)
    headers = {
    "Accept": "application/json",
    "Authorization": "fsq3daN1NA7XlU6QvMrgyqCe4mLWWXHDbhJ5uoO2cEPfepA="}
    url = 'https://api.foursquare.com/v3/places/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        Latitude,
        Longitude, 
        radius, 
       LIMIT)
    new_results = requests.get(url,headers=headers).json()
    (results['results'].extend(new_results['results']))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [79]:
len(results['results'])

8817

In [80]:
results['results']

[{'fsq_id': '4b2a6eb8f964a52012a924e3',
  'categories': [{'id': 17018,
    'name': 'Bookstore',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/bookstore_',
     'suffix': '.png'}}],
  'chains': [{'id': 'a050b295-4a43-4d68-ab40-2df63d9c90ab', 'name': 'Indigo'}],
  'distance': 204,
  'geocodes': {'main': {'latitude': 43.653755, 'longitude': -79.380001},
   'roof': {'latitude': 43.653755, 'longitude': -79.380001}},
  'link': '/v3/places/4b2a6eb8f964a52012a924e3',
  'location': {'address': '220 Yonge St',
   'address_extended': '# 300',
   'country': 'CA',
   'cross_street': '',
   'formatted_address': '220 Yonge St, Toronto ON M5B 2H1',
   'locality': 'Toronto',
   'neighborhood': ['Eaton Centre'],
   'postcode': 'M5B 2H1',
   'region': 'ON'},
  'name': 'Indigo',
  'related_places': {'parent': {'fsq_id': '4ad77a12f964a520260b21e3',
    'name': 'CF Toronto Eaton Centre'},
   'children': []},
  'timezone': 'America/Toronto'},
 {'fsq_id': '506db1a9e4b0a3f3b31412f0',
  '

In [81]:
br_df = pd.DataFrame.from_dict(results['results'], orient='columns')
br_df=br_df.drop(columns=['fsq_id','chains','link','related_places','timezone','distance'])

In [82]:
br_df

,categories,geocodes,location,name
0,"[{'id': 17018, 'name': 'Bookstore', 'icon': {'...","{'main': {'latitude': 43.653755, 'longitude': ...","{'address': '220 Yonge St', 'address_extended'...",Indigo
1,"[{'id': 13068, 'name': 'American Restaurant', ...","{'main': {'latitude': 43.65151, 'longitude': -...","{'address': '1 Richmond St W', 'country': 'CA'...",Richmond Station
2,"[{'id': 10039, 'name': 'Music Venue', 'icon': ...","{'main': {'latitude': 43.650708, 'longitude': ...","{'address': '145 Queen St W', 'country': 'CA',...",The Four Seasons Centre for the Performing Arts
3,"[{'id': 17043, 'name': 'Clothing Store', 'icon...","{'main': {'latitude': 43.655961, 'longitude': ...","{'address': '220 Yonge St', 'country': 'CA', '...",Uniqlo ユニクロ
4,"[{'id': 10039, 'name': 'Music Venue', 'icon': ...","{'main': {'latitude': 43.653129, 'longitude': ...","{'address': '189 Yonge St', 'country': 'CA', '...",Elgin & Winter Garden Theatre Ctr
...,...,...,...,...
8812,"[{'id': 10024, 'name': 'Movie Theater', 'icon'...","{'main': {'latitude': 43.722421, 'longitude': ...","{'address': '22 Lebovic Ave', 'country': 'CA',...",Cineplex Odeon Cinemas
8813,"[{'id': 17076, 'name': 'Liquor Store', 'icon':...","{'main': {'latitude': 43.735507, 'longitude': ...","{'address': '510 Brimley Rd', 'country': 'CA',...",Lcbo
8814,"[{'id': 17034, 'name': 'Discount Store', 'icon...","{'main': {'latitude': 43.727831, 'longitude': ...","{'address': '1920 Eglinton Ave E', 'country': ...",Dollarama
8815,"[{'id': 13145, 'name': 'Fast Food Restaurant',...","{'main': {'latitude': 43.734671, 'longitude': ...","{'address': '2567 Eglinton Ave E', 'country': ...",KFC


In [83]:
places = br_df[['name', 'categories', 'location', 'geocodes']]

In [84]:
places

,name,categories,location,geocodes
0,Indigo,"[{'id': 17018, 'name': 'Bookstore', 'icon': {'...","{'address': '220 Yonge St', 'address_extended'...","{'main': {'latitude': 43.653755, 'longitude': ..."
1,Richmond Station,"[{'id': 13068, 'name': 'American Restaurant', ...","{'address': '1 Richmond St W', 'country': 'CA'...","{'main': {'latitude': 43.65151, 'longitude': -..."
2,The Four Seasons Centre for the Performing Arts,"[{'id': 10039, 'name': 'Music Venue', 'icon': ...","{'address': '145 Queen St W', 'country': 'CA',...","{'main': {'latitude': 43.650708, 'longitude': ..."
3,Uniqlo ユニクロ,"[{'id': 17043, 'name': 'Clothing Store', 'icon...","{'address': '220 Yonge St', 'country': 'CA', '...","{'main': {'latitude': 43.655961, 'longitude': ..."
4,Elgin & Winter Garden Theatre Ctr,"[{'id': 10039, 'name': 'Music Venue', 'icon': ...","{'address': '189 Yonge St', 'country': 'CA', '...","{'main': {'latitude': 43.653129, 'longitude': ..."
...,...,...,...,...
8812,Cineplex Odeon Cinemas,"[{'id': 10024, 'name': 'Movie Theater', 'icon'...","{'address': '22 Lebovic Ave', 'country': 'CA',...","{'main': {'latitude': 43.722421, 'longitude': ..."
8813,Lcbo,"[{'id': 17076, 'name': 'Liquor Store', 'icon':...","{'address': '510 Brimley Rd', 'country': 'CA',...","{'main': {'latitude': 43.735507, 'longitude': ..."
8814,Dollarama,"[{'id': 17034, 'name': 'Discount Store', 'icon...","{'address': '1920 Eglinton Ave E', 'country': ...","{'main': {'latitude': 43.727831, 'longitude': ..."
8815,KFC,"[{'id': 13145, 'name': 'Fast Food Restaurant',...","{'address': '2567 Eglinton Ave E', 'country': ...","{'main': {'latitude': 43.734671, 'longitude': ..."


Getting category title

In [85]:
br_df['categories'][0][0]['name']

'Bookstore'

In [104]:
name_list2 = []
name2 = []
for i in range(len(br_df)):
    name2 = []
    for j in range(len(br_df['categories'][i])):
        name2.append(br_df['categories'][i][j]['name'])
        #print(br_df['categories'][i][j]['name'])
    if not name2:
        pass
    else:
        name_list2.append(name2[0])

In [112]:
name_list = []
name = []
for i in range(len(br_df)):
    name = []
    for j in range(len(br_df['categories'][i])):
        name.append(br_df['categories'][i][j]['name'])
    if not name:
        name_list.append("Restaurant")
    else:
        name_list.append(name[0])

In [113]:
name_list[0]

'Bookstore'

In [114]:
places['Category']= name_list

<ipython-input-114-285d97d98991>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  places['Category']= name_list


In [115]:
places.head()

,name,categories,location,geocodes,Category
0,Indigo,"[{'id': 17018, 'name': 'Bookstore', 'icon': {'...","{'address': '220 Yonge St', 'address_extended'...","{'main': {'latitude': 43.653755, 'longitude': ...",Bookstore
1,Richmond Station,"[{'id': 13068, 'name': 'American Restaurant', ...","{'address': '1 Richmond St W', 'country': 'CA'...","{'main': {'latitude': 43.65151, 'longitude': -...",American Restaurant
2,The Four Seasons Centre for the Performing Arts,"[{'id': 10039, 'name': 'Music Venue', 'icon': ...","{'address': '145 Queen St W', 'country': 'CA',...","{'main': {'latitude': 43.650708, 'longitude': ...",Music Venue
3,Uniqlo ユニクロ,"[{'id': 17043, 'name': 'Clothing Store', 'icon...","{'address': '220 Yonge St', 'country': 'CA', '...","{'main': {'latitude': 43.655961, 'longitude': ...",Clothing Store
4,Elgin & Winter Garden Theatre Ctr,"[{'id': 10039, 'name': 'Music Venue', 'icon': ...","{'address': '189 Yonge St', 'country': 'CA', '...","{'main': {'latitude': 43.653129, 'longitude': ...",Music Venue


Getting postal codes of all places

In [116]:
br_df['location'][0]['postcode'].split()[0]

'M5B'

In [117]:
import numpy as np
for row in br_df['location'].iteritems():
    if not 'postcode' in row[1].keys():
        row[1]['postcode']=np.random.choice(df['PostalCode'])
    else:
        if(row[1]['postcode']==""):
            row[1]['postcode']=np.random.choice(df['PostalCode'])

In [118]:
for i in range(len(br_df)):
    location = []
    for j in range(len(br_df['location'][i])):
        if(not br_df['location'][i]['postcode']):
            print(br_df['location'][i])

In [119]:
postalCode = []
location = []
for i in range(len(br_df)):
    location = []
    for j in range(len(br_df['location'][i])):
        if(br_df['location'][i]['postcode']):
            location.append(br_df['location'][i]['postcode'].split()[0])
        else:
            location.append(br_df['location'][i]['postcode'])
        
    if(location):
        postalCode.append(location[0])
    else:
        postalCode.append(location)

In [120]:
len(postalCode)

8817

In [121]:
places['PostalCode']= postalCode

<ipython-input-121-74a871219b22>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  places['PostalCode']= postalCode


In [122]:
places = places.drop(columns=['categories','location','geocodes'])

In [123]:
places.head()

,name,Category,PostalCode
0,Indigo,Bookstore,M5B
1,Richmond Station,American Restaurant,M5H
2,The Four Seasons Centre for the Performing Arts,Music Venue,M5H
3,Uniqlo ユニクロ,Clothing Store,M5B
4,Elgin & Winter Garden Theatre Ctr,Music Venue,M5B


In [124]:
newdf.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [125]:
final_df=pd.merge(places, newdf, on="PostalCode")

In [126]:
final_df=final_df.rename(columns={'name':'Place Name'})

In [127]:
len(final_df)

7204

In [128]:
len(final_df['Place Name'].unique())

2399

Removing duplicates from data based on the place name column.

In [130]:
final_df =final_df.drop_duplicates(subset=['Place Name'],keep=False)

In [131]:
#final_df =final_df.drop_duplicates(subset=['Place Name','Category'],keep=False)

In [132]:
final_df

,Place Name,Category,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,Elgin & Winter Garden Theatre Ctr,Music Venue,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
11,Carville Mill Park,Park,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
23,Hina's Beauty Care,Hair Salon,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
34,Richmond Station,American Restaurant,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
44,Cactus Club Cafe,American Restaurant,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
...,...,...,...,...,...,...,...
7123,Exchange cafe,Café,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
7183,Melmira Bra & Swimsuits Inc,Clothing Store,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
7192,Boom Breakfast & Co,Diner,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7193,Burger Shack,Burger Joint,M4R,Central Toronto,North Toronto West,43.715383,-79.405678


Storing the file with the name data.csv on the local machine.

In [133]:
final_df.to_csv('data.csv',encoding='utf-8')

In [134]:
final_df.shape

(954, 7)